In [1]:
import os
import sys
import numpy as np
import neuron
from neuron import h
from neuron.units import ms, mV
from neuron import gui
from matplotlib import pyplot as plt
import csv
from math import exp, log
import scipy
import seaborn as sns

h.load_file("import3d.hoc")
morphology_file = "morphologies/cell1.asc"
h.load_file("L5PCbiophysWhat.hoc")
h.load_file("L5PCtemplate_2.hoc")
L5PC = h.L5PCtemplate(morphology_file)
h.celsius = 37

845.34678 
845.34678 
845.34678 


In [2]:
apic_secs = list(L5PC.all)
for sec in apic_secs:
    for seg in sec:
        try: seg.Ih.vshift = seg.Ih.vshift -15
        except AttributeError: continue   
for sec in apic_secs:
    for seg in sec:
        try: seg.Ca_HVA.gCa_HVAbar = seg.Ca_HVA.gCa_HVAbar*0.3
        except AttributeError: continue   
c = 1
for i in range (0,30):   
    START_TIME = 25     # time to start the stimulus
    END_TIME = 3000     # time to end the stimulus
    TIME_PER_AMP = 5    # change amplitude every TIME_PER_AMP ms
    MEAN_AMP = 0.15     # the mean amplitude
    LAMBDA = 0.5        # the poisson mean
    MEAN_AMP_2 = 0.15      # the mean amplitude
    LAMBDA_2 = 3        # the poisson mean
    times = np.arange(START_TIME, END_TIME, TIME_PER_AMP)
    totime = END_TIME-START_TIME

    random_amplitude = 1 / (np.random.poisson(LAMBDA, times.shape[0]) + 1) \
                       - 1 / (np.random.poisson(LAMBDA, times.shape[0]) + 1)
    random_amplitude_2 = 1 / (np.random.poisson(LAMBDA_2, times.shape[0]) + 1) \
                       - 1 / (np.random.poisson(LAMBDA_2, times.shape[0]) + 1)
    random_amplitude += MEAN_AMP
    random_amplitude_2 += MEAN_AMP_2
    clamps = []
    clamps_records = []
    clamps2 = []
    clamps_records2 = []
    for t in range(times.shape[0]):
        stim = h.IClamp(L5PC.soma[0](0.5))
        stim.delay = times[t]
        stim.amp = random_amplitude[t]
        stim.dur = TIME_PER_AMP
        clamps.append(stim)
        clamps_records.append(h.Vector().record(stim._ref_i))

        stim2 = h.IClamp(L5PC.apic[36](0.9))
        stim2.delay = times[t]
        stim2.amp = random_amplitude_2[t]
        stim2.dur = TIME_PER_AMP
        clamps2.append(stim2)
        clamps_records2.append(h.Vector().record(stim2._ref_i))

    neuron_times = h.Vector().record(h._ref_t)
    threshold = -20
    som_v = h.Vector().record(L5PC.soma[0](0.5)._ref_v)
    distal_v = h.Vector().record(L5PC.apic[36](0.9)._ref_v)
    distal_ca = h.Vector().record(L5PC.apic[36](0.9).Ca_HVA._ref_ica)
    count = 0
    h.finitialize(-80*mV)
    h.continuerun(END_TIME + 50)
    stimulus_as_one_array = np.sum([np.array(rec) for rec in clamps_records], axis=0)
    stimulus_as_one_array_2 = np.sum([np.array(rec) for rec in clamps_records2], axis=0)
    sumi = stimulus_as_one_array + stimulus_as_one_array_2

    arr = np.array(distal_v) > -5
    second_arr = arr[:-1]
    arr = arr[1:]
    arr1 = np.array(som_v) > 0
    second_arr1 = arr1[:-1]
    arr1 = arr1[1:]
    #return np.sum(np.logical_and(arr, np.logical_not(second_arr)))
    counts = np.sum(np.logical_and(arr1, np.logical_not(second_arr1)))
    countp = np.sum(np.logical_and(arr, np.logical_not(second_arr)))
    print(c, counts, countp)
    c+=1


1 24 2
2 45 1
3 49 2
4 35 3
5 32 1
6 32 1
7 27 1


KeyboardInterrupt: 